In [17]:
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
matplotlib.rcParams["figure.figsize"] = (16,9)

In [18]:
df = pd.read_csv("usa_rain_prediction_dataset_2024_2025.csv")
df

,Date,Location,Temperature,Humidity,Wind Speed,Precipitation,Cloud Cover,Pressure,Rain Tomorrow
0,2024-01-01,New York,87.524795,75.655455,28.379506,0.000000,69.617966,1026.030278,0
1,2024-01-02,New York,83.259325,28.712617,12.436433,0.526995,41.606048,995.962065,0
2,2024-01-03,New York,80.943050,64.740043,14.184831,0.916884,77.364763,980.796739,1
3,2024-01-04,New York,78.097552,59.738984,19.444029,0.094134,52.541196,979.012163,0
4,2024-01-05,New York,37.059963,34.766784,3.689661,1.361272,85.584000,1031.790859,0
...,...,...,...,...,...,...,...,...,...
73095,2025-12-27,Washington D.C.,40.614393,65.099438,28.778327,0.000000,54.168514,977.083747,0
73096,2025-12-28,Washington D.C.,52.641643,30.610525,12.282890,0.871000,22.068055,980.591675,0
73097,2025-12-29,Washington D.C.,56.492591,96.740232,2.894762,1.191956,52.336048,1016.469174,1
73098,2025-12-30,Washington D.C.,65.748956,63.900004,24.632400,0.483421,76.785280,1032.396146,1


In [19]:
df["Date"] = pd.to_datetime(df["Date"])

In [20]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 73100 entries, 0 to 73099
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   Date           73100 non-null  datetime64[ns]
 1   Location       73100 non-null  object        
 2   Temperature    73100 non-null  float64       
 3   Humidity       73100 non-null  float64       
 4   Wind Speed     73100 non-null  float64       
 5   Precipitation  73100 non-null  float64       
 6   Cloud Cover    73100 non-null  float64       
 7   Pressure       73100 non-null  float64       
 8   Rain Tomorrow  73100 non-null  int64         
dtypes: datetime64[ns](1), float64(6), int64(1), object(1)
memory usage: 5.0+ MB


In [21]:
df.isna().sum()

Date             0
Location         0
Temperature      0
Humidity         0
Wind Speed       0
Precipitation    0
Cloud Cover      0
Pressure         0
Rain Tomorrow    0
dtype: int64

In [22]:
df.describe()

,Date,Temperature,Humidity,Wind Speed,Precipitation,Cloud Cover,Pressure,Rain Tomorrow
count,73100,73100.000000,73100.000000,73100.000000,73100.000000,73100.000000,73100.000000,73100.000000
mean,2024-12-31 00:00:00,65.182270,59.875041,15.017946,0.390635,54.942807,1005.176013,0.220410
min,2024-01-01 00:00:00,30.000766,20.000272,0.000712,0.000000,10.000856,970.000919,0.000000
25%,2024-07-01 00:00:00,47.678968,39.800732,7.485182,0.000000,32.318668,987.697646,0.000000
50%,2024-12-31 00:00:00,65.294949,59.887840,15.102495,0.196909,55.011121,1005.284188,0.000000
75%,2025-07-02 00:00:00,82.636570,79.835990,22.551794,0.673177,77.412469,1022.727410,0.000000
max,2025-12-31 00:00:00,99.997393,99.999025,29.999017,3.078090,99.998957,1039.999765,1.000000
std,NaN,20.205793,23.066115,8.668729,0.474833,25.982487,20.203889,0.414526


In [23]:
# fix preciptation, wind speed outlier
def removeOutlier(df, cols):
    for col in cols:
        q3, q1 = df[col].quantile([0.75, 0.25])
        iqr = q3 - q1
        df = df[(df[col] > q1 - 1.5 * iqr) & (df[col] < q3 + 1.5 * iqr)]
    return df

In [24]:
cols = ["Wind Speed", "Precipitation"]
df2 = removeOutlier(df, cols)

In [25]:
df3 = df2.copy()

In [26]:
from sklearn.preprocessing import LabelEncoder

In [27]:
df3

,Date,Location,Temperature,Humidity,Wind Speed,Precipitation,Cloud Cover,Pressure,Rain Tomorrow
0,2024-01-01,New York,87.524795,75.655455,28.379506,0.000000,69.617966,1026.030278,0
1,2024-01-02,New York,83.259325,28.712617,12.436433,0.526995,41.606048,995.962065,0
2,2024-01-03,New York,80.943050,64.740043,14.184831,0.916884,77.364763,980.796739,1
3,2024-01-04,New York,78.097552,59.738984,19.444029,0.094134,52.541196,979.012163,0
4,2024-01-05,New York,37.059963,34.766784,3.689661,1.361272,85.584000,1031.790859,0
...,...,...,...,...,...,...,...,...,...
73095,2025-12-27,Washington D.C.,40.614393,65.099438,28.778327,0.000000,54.168514,977.083747,0
73096,2025-12-28,Washington D.C.,52.641643,30.610525,12.282890,0.871000,22.068055,980.591675,0
73097,2025-12-29,Washington D.C.,56.492591,96.740232,2.894762,1.191956,52.336048,1016.469174,1
73098,2025-12-30,Washington D.C.,65.748956,63.900004,24.632400,0.483421,76.785280,1032.396146,1


In [28]:
# extract datetime data
df3["day"] =  df3.Date.dt.day
df3["month"] =  df3.Date.dt.month
df3["year"] =  df3.Date.dt.year
df3.drop(["Date"], axis=1, inplace=True)

In [29]:
le = LabelEncoder()
df3["Location"] = le.fit_transform(df3["Location"])

In [30]:
df3

,Location,Temperature,Humidity,Wind Speed,Precipitation,Cloud Cover,Pressure,Rain Tomorrow,day,month,year
0,11,87.524795,75.655455,28.379506,0.000000,69.617966,1026.030278,0,1,1,2024
1,11,83.259325,28.712617,12.436433,0.526995,41.606048,995.962065,0,2,1,2024
2,11,80.943050,64.740043,14.184831,0.916884,77.364763,980.796739,1,3,1,2024
3,11,78.097552,59.738984,19.444029,0.094134,52.541196,979.012163,0,4,1,2024
4,11,37.059963,34.766784,3.689661,1.361272,85.584000,1031.790859,0,5,1,2024
...,...,...,...,...,...,...,...,...,...,...,...
73095,19,40.614393,65.099438,28.778327,0.000000,54.168514,977.083747,0,27,12,2025
73096,19,52.641643,30.610525,12.282890,0.871000,22.068055,980.591675,0,28,12,2025
73097,19,56.492591,96.740232,2.894762,1.191956,52.336048,1016.469174,1,29,12,2025
73098,19,65.748956,63.900004,24.632400,0.483421,76.785280,1032.396146,1,30,12,2025


In [31]:
x = df3.drop(["Rain Tomorrow"], axis=1)
y = df3[["Rain Tomorrow"]]

In [32]:
from sklearn.model_selection import train_test_split

In [33]:
x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=69, test_size=0.2)

In [34]:
from sklearn.tree import DecisionTreeClassifier

In [35]:
tree = DecisionTreeClassifier()
tree.fit(x_train, y_train)

DecisionTreeClassifier()

In [36]:
tree.score(x_test, y_test)

1.0

In [37]:
df3["pred"] = tree.predict(x)

In [38]:
df3

,Location,Temperature,Humidity,Wind Speed,Precipitation,Cloud Cover,Pressure,Rain Tomorrow,day,month,year,pred
0,11,87.524795,75.655455,28.379506,0.000000,69.617966,1026.030278,0,1,1,2024,0
1,11,83.259325,28.712617,12.436433,0.526995,41.606048,995.962065,0,2,1,2024,0
2,11,80.943050,64.740043,14.184831,0.916884,77.364763,980.796739,1,3,1,2024,1
3,11,78.097552,59.738984,19.444029,0.094134,52.541196,979.012163,0,4,1,2024,0
4,11,37.059963,34.766784,3.689661,1.361272,85.584000,1031.790859,0,5,1,2024,0
...,...,...,...,...,...,...,...,...,...,...,...,...
73095,19,40.614393,65.099438,28.778327,0.000000,54.168514,977.083747,0,27,12,2025,0
73096,19,52.641643,30.610525,12.282890,0.871000,22.068055,980.591675,0,28,12,2025,0
73097,19,56.492591,96.740232,2.894762,1.191956,52.336048,1016.469174,1,29,12,2025,1
73098,19,65.748956,63.900004,24.632400,0.483421,76.785280,1032.396146,1,30,12,2025,1
